In [4]:
import sys, pathlib
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Setup ruta al proyecto
PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg

# === 1. Cargar datos ===
data = joblib.load(cfg.DATA / "processed" / "cnn5d_data.pkl")
X, y = data["X"], data["y"]
fechas = data["dates"]

In [18]:
# === 2. División temporal ===
train_mask = fechas < "2019-01-01"
val_mask   = (fechas >= "2019-01-01") & (fechas < "2021-01-01")
test_mask  = fechas >= "2021-01-01"

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val     = X[val_mask], y[val_mask]
X_test, y_test   = X[test_mask], y[test_mask]

print(f"🔹 Train: {X_train.shape}")
print(f"🔹 Val:   {X_val.shape}")
print(f"🔹 Test:  {X_test.shape}")
print("🗓️ Fechas:", fechas.min(), "→", fechas.max())

print("📅 Rango de fechas por conjunto:")
print("Train:", fechas[train_mask].min(), "→", fechas[train_mask].max())
print("Val:  ", fechas[val_mask].min(), "→", fechas[val_mask].max())
print("Test: ", fechas[test_mask].min(), "→", fechas[test_mask].max())

🔹 Train: (2086, 60, 80)
🔹 Val:   (731, 60, 80)
🔹 Test:  (1633, 60, 80)
🗓️ Fechas: 2012-08-22 00:00:00 → 2025-06-21 00:00:00
📅 Rango de fechas por conjunto:
Train: 2012-08-22 00:00:00 → 2018-12-31 00:00:00
Val:   2019-01-01 00:00:00 → 2020-12-31 00:00:00
Test:  2021-01-01 00:00:00 → 2025-06-21 00:00:00


In [8]:
# === 3. Escalado ===
scaler_X = StandardScaler()
X_train_2d = X_train.reshape(-1, X.shape[2])
X_train_scaled = scaler_X.fit_transform(X_train_2d).reshape(X_train.shape)

X_val_scaled = scaler_X.transform(X_val.reshape(-1, X.shape[2])).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X.shape[2])).reshape(X_test.shape)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)

joblib.dump(scaler_X, cfg.MODELS / "scaler_X_cnn5d.pkl")
joblib.dump(scaler_y, cfg.MODELS / "scaler_y_cnn5d.pkl")

print("✅ X escalado correctamente:", X_train_scaled.shape)



✅ X escalado correctamente: (2086, 60, 80)


In [10]:
# === 4. Definir modelo CNN ===
model = models.Sequential([
    layers.Input(shape=(cfg.WINDOW, X.shape[2])),
    layers.Conv1D(32, kernel_size=3, padding="causal", activation="relu"),
    layers.Conv1D(32, kernel_size=3, padding="causal", activation="relu"),
    layers.GlobalAveragePooling1D(),
    layers.Dense(y.shape[1])
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
model.summary()

# === 5. Entrenamiento ===
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 60, 32)         │         7,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 60, 32)         │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │         1,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,136 (47.41 KB)

 Trainable params: 12,136 (47.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.0362 - val_loss: 2.0384
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0048 - val_loss: 2.0402
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9740 - val_loss: 2.0378
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0034 - val_loss: 2.0375
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9782 - val_loss: 2.0450
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9588 - val_loss: 2.0448
Epoch 7/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9427 - val_loss: 2.0584
Epoch 8/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9186 - val_loss: 2.0846
Epoch 9/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9537 - val_loss: 2.0962


In [12]:
# === 6. Guardado ===
model.save(cfg.MODELS / "cnn5d.keras")
joblib.dump(history.history, cfg.RESULT / "history_cnn5d.pkl")

# === 7. Evaluación ===
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(((y_test_scaled - y_pred)**2).mean(axis=0))
rmse_mean = rmse.mean()
print("📉 RMSE medio:", rmse_mean)

joblib.dump(rmse_mean, cfg.RESULT / "rmse_cnn5d.pkl")

52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
📉 RMSE medio: 1.2003729


['C:\\Users\\ferra\\Documents\\TFM\\results\\rmse_cnn5d.pkl']